## ARTIFICIAL NEURAL NETWORKS


Classification Using Artificial Neural Networks with Hyperparameter Tuning on Alphabets Data



In [1]:
 %pip install tensorflow


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('Alphabets_data.csv')

In [4]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [5]:
def preprocess_data(df):
    # Encode target variable
    le = LabelEncoder()
    df['letter'] = le.fit_transform(df['letter'])

    # Split features and target variable
    X = df.drop(columns=['letter'])
    y = df['letter']

    # Standardize the features
    sc = StandardScaler()
    X = sc.fit_transform(X)

    return X, y

In [6]:
# Function to build the model
def build_model():
    model = Sequential()
    model.add(Dense(units=10, activation='relu', input_shape=(16,)))  # Input shape based on number of features
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# Function to tune hyperparameters
def tune_hyperparameters(X_train, y_train, X_test, y_test):
    import keras_tuner as kt

    def optimizer_selection(hp):
        model = Sequential()
        model.add(Dense(units=10, activation='relu', input_shape=(16,)))
        model.add(Dense(units=1, activation='sigmoid'))

        optim = hp.Choice('optimizer', values=['sgd', 'adam', 'rmsprop'])
        model.compile(optimizer=optim, loss='binary_crossentropy', metrics=['accuracy'])
        return model

    tuner = kt.RandomSearch(
        optimizer_selection,
        objective='val_accuracy',
        max_trials=3
    )
    tuner.search(X_train, y_train, epochs=3, validation_data=(X_test, y_test))
    return tuner.get_best_hyperparameters()[0].values['optimizer']



# Data Exploration
print(df.describe())
print(df.head())
print(df.tail())
print(df.isna().sum())

               xbox          ybox         width       height         onpix  \
count  20000.000000  20000.000000  20000.000000  20000.00000  20000.000000   
mean       4.023550      7.035500      5.121850      5.37245      3.505850   
std        1.913212      3.304555      2.014573      2.26139      2.190458   
min        0.000000      0.000000      0.000000      0.00000      0.000000   
25%        3.000000      5.000000      4.000000      4.00000      2.000000   
50%        4.000000      7.000000      5.000000      6.00000      3.000000   
75%        5.000000      9.000000      6.000000      7.00000      5.000000   
max       15.000000     15.000000     15.000000     15.00000     15.000000   

               xbar          ybar         x2bar         y2bar         xybar  \
count  20000.000000  20000.000000  20000.000000  20000.000000  20000.000000   
mean       6.897600      7.500450      4.628600      5.178650      8.282050   
std        2.026035      2.325354      2.699968      2.38082

In [8]:
# Preprocess data
X, y = preprocess_data(df)

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
# Build and train the model
model = build_model()
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0367 - loss: -13.6038
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0392 - loss: -13.7985  
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0394 - loss: -13.9274
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0376 - loss: -13.8896
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0400 - loss: -14.3398  
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0386 - loss: -14.4496  
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0365 - loss: -14.4222
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0396 - loss: -14.6664
Epoch 9/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0415 - loss: -14.6007  
Epoch 10/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0403 - loss: -14.7305
Epoch 11/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0379 - loss: -15.1671
Epoch 12/10

In [11]:

# Predict on test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary output

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [20]:
# Evaluate model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       162
           1       0.04      1.00      0.07       152
           2       0.00      0.00      0.00       144
           3       0.00      0.00      0.00       156
           4       0.00      0.00      0.00       139
           5       0.00      0.00      0.00       144
           6       0.00      0.00      0.00       154
           7       0.00      0.00      0.00       146
           8       0.00      0.00      0.00       138
           9       0.00      0.00      0.00       141
          10       0.00      0.00      0.00       137
          11       0.00      0.00      0.00       139
          12       0.00      0.00      0.00       145
          13       0.00      0.00      0.00       161
          14       0.00      0.00      0.00       173
          15       0.00      0.00      0.00       174
          16       0.00      0.00      0.00       159
          17       0.00    

In [13]:
import keras_tuner

In [14]:
%pip install -U keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [18]:
# Hyperparameter Tuning
best_optimizer = tune_hyperparameters(X_train, y_train, X_test, y_test)
print(f"Best optimizer: {best_optimizer}")

Reloading Tuner from .\untitled_project\tuner0.json
Best optimizer: sgd


In [21]:
# Retrain model with the best hyperparameters
model = build_model()
model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0366 - loss: nan - val_accuracy: 0.0380 - val_loss: nan
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0406 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0370 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0369 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0400 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0395 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0381 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0422 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 9/100
500/

In [17]:
# Evaluate the final model
evaluation = model.evaluate(X_train, y_train)
print(f"Final Model - Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0380 - loss: nan
Final Model - Loss: nan, Accuracy: 0.03918749839067459


## Conclusion :

The code has been refactored for clarity and modularity, with functions for loading data, preprocessing, building models, and tuning hyperparameters. It ensures better organization, improved handling of predictions, and efficient hyperparameter tuning for optimizing model performance.